In [1]:
!pip install imbalanced-learn
!pip install scikit-learn
!pip install gdown
!pip install pandas
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
anyio                     4.3.0
appnope                   0.1.4
archspec                  0.2.1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
astunparse                1.6.3
async-lru                 2.0.4
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.3
bleach                    6.1.0
boltons                   23.0.0
Brotli                    1.0.9
certifi                   2024.2.2
cffi                      1.16.0
charset-normalizer        2.0.4
comm                      0.2.2
conda                     24.1.2
conda-content-trust       0.2.0
conda-libmamba-solver     23.12.0
conda-package-handling    2.2.0
conda_package_streaming   0.9.0
contourpy                 1.2.1
cryptography              41.0.7
cycler                    0.12.1
debugpy       

In [3]:
# Required Libraries
import gdown
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
isFileExist = list(Path('.').glob('*.csv'))
if isFileExist:
    print('Datasets are already available - No downloading')
else:
    file_ids = [
        '1zOeCqtGZjAj_nSLe3W5MH5pDQM28bPde',  # Replace with actual file ID
        '1fagr2rKW8EN-Psc5UlD9BFfa0rRRF6i3',  # Replace with actual file ID
        '1WL00LFkCA2ylV8_8c6jm52bnEhGLcKsq',   # Replace with actual file ID
        '1xr_QBU3Ab42nW1ELl0NUydx4Rbx60TVR',
        '1MpqHsC5wQSu9CxlhJimz25EVcYubSMhu',
        '18mKfJHXinmfwPpaYNU2D6xHwq4dQuEuL',
        '1h1lMHytamf4Kc66NfAfMfBzDbmwM5Ub5',
        '11__JhC64_D4ezxjF8d0dPNRDJF-TY6Pq'
    ]    
    # Download each file
    for i, file_id in enumerate(file_ids):
        url = f'https://drive.google.com/uc?id={file_id}'
        gdown.download(url, f'dataset_{i + 1}.csv', quiet=False)  # Saves as dataset_1.csv, dataset_2.csv, etc.

Datasets are already available - No downloading


In [5]:
dataset_paths = [
    './dataset_1.csv','./dataset_2.csv','./dataset_3.csv','./dataset_4.csv',
    './dataset_5.csv','./dataset_6.csv','./dataset_7.csv','./dataset_8.csv',
]

# Make the data frame
dataframes = [pd.read_csv(path) for path in dataset_paths]

# Combine all datasets into one dataframe
combined_dataset = pd.concat(dataframes, ignore_index=True)

In [6]:
combined_dataset.describe()

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,...,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,8.071483e+03,1.478566e+07,9.361160e+00,1.039377e+01,5.493024e+02,1.616264e+04,2.075999e+02,1.871366e+01,5.820194e+01,6.891013e+01,...,5.418218e+00,-2.741688e+03,8.155132e+04,4.113412e+04,1.531825e+05,5.829582e+04,8.316037e+06,5.038439e+05,8.695752e+06,7.920031e+06
std,1.828363e+04,3.365374e+07,7.496728e+02,9.973883e+02,9.993589e+03,2.263088e+06,7.171848e+02,6.033935e+01,1.860912e+02,2.811871e+02,...,6.364257e+02,1.084989e+06,6.485999e+05,3.933815e+05,1.025825e+06,5.770923e+05,2.363008e+07,4.602984e+06,2.436689e+07,2.336342e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,1.550000e+02,2.000000e+00,1.000000e+00,1.200000e+01,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,3.131600e+04,2.000000e+00,2.000000e+00,6.200000e+01,1.230000e+02,3.700000e+01,2.000000e+00,3.400000e+01,0.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,3.204828e+06,5.000000e+00,4.000000e+00,1.870000e+02,4.820000e+02,8.100000e+01,3.600000e+01,5.000000e+01,2.616295e+01,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,2.135570e+05,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


In [7]:
# Pre processing
x = combined_dataset.drop(columns=[' Label'])
y = combined_dataset[' Label']
# Handle missing values
x.fillna(x.mean(), inplace=True)

# Get 'Label' column that indicates attack types
class_distribution = combined_dataset[' Label'].value_counts()
print(class_distribution)

 Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [8]:
#selecting best features for the training model out of 78 features
#use Random forest algorithm
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

#Train a model with Random Forest
x.replace([float('inf'), float('-inf')], float('nan'), inplace=True)
rfModel=RandomForestClassifier(n_estimators=100, random_state=42)
rfModel.fit(x,y)

#Get important feature
importantFeatures=rfModel.feature_importances_

#Creating another Data frame with selected most importand features
importantFeatures_df=pd.DataFrame({
    'Feature' : x.columns,
    'Importance': importantFeatures
})

#Sort the Dataframe by importance scores in decending order
importantFeatures_df = importantFeatures_df.sort_values(by='Importance', ascending=False)
# Display the top important features
print(importantFeatures_df.head(20))
'''
# Plotting feature importances
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.barh(importantFeatures_df['Feature'][:20], importantFeatures_df['Importance'][:20], color='skyblue')  # Change to top 20
plt.xlabel('Importance Score')
plt.title('Top 20 Important Features')  # Update title to reflect the change
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top
plt.show()
'''

                         Feature  Importance
41             Packet Length Std    0.054740
42        Packet Length Variance    0.053101
39             Max Packet Length    0.048593
52           Average Packet Size    0.042901
54          Avg Bwd Segment Size    0.042895
0               Destination Port    0.040938
63             Subflow Fwd Bytes    0.037041
10         Bwd Packet Length Max    0.035396
5    Total Length of Bwd Packets    0.034872
13         Bwd Packet Length Std    0.031664
4    Total Length of Fwd Packets    0.030893
66        Init_Win_bytes_forward    0.028209
6          Fwd Packet Length Max    0.026371
65             Subflow Bwd Bytes    0.026302
53          Avg Fwd Segment Size    0.024739
12        Bwd Packet Length Mean    0.024135
40            Packet Length Mean    0.023360
55           Fwd Header Length.1    0.019171
46                PSH Flag Count    0.017399
8         Fwd Packet Length Mean    0.016899


"\n# Plotting feature importances\nplt.figure(figsize=(10, 6))  # Adjust the figure size as needed\nplt.barh(importantFeatures_df['Feature'][:20], importantFeatures_df['Importance'][:20], color='skyblue')  # Change to top 20\nplt.xlabel('Importance Score')\nplt.title('Top 20 Important Features')  # Update title to reflect the change\nplt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top\nplt.show()\n"

In [9]:
# Update x
top_20_features = importantFeatures_df['Feature'].head(20).values
x = x[top_20_features]  # updates x directly

'''
if isinstance(x, pd.DataFrame):
    x['Label'] = combined_dataset[' Label'].loc[x.index].values  # Align indices
else:
    raise ValueError("x is not a DataFrame.")

print("Updated x with 'Label':")
#print(x.head())  # Display the first few rows of the updated DataFrame
print(x['Label'].value_counts())
'''

'\nif isinstance(x, pd.DataFrame):\n    x[\'Label\'] = combined_dataset[\' Label\'].loc[x.index].values  # Align indices\nelse:\n    raise ValueError("x is not a DataFrame.")\n\nprint("Updated x with \'Label\':")\n#print(x.head())  # Display the first few rows of the updated DataFrame\nprint(x[\'Label\'].value_counts())\n'

In [10]:
import time
start_time = time.time()

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42, stratify=y)
x_train.replace([float('inf'), float('-inf')], float('nan'), inplace=True)
x_test.replace([float('inf'), float('-inf')], float('nan'), inplace=True)

x_train.fillna(x_train.mean(), inplace=True)
x_test.fillna(x_test.mean(), inplace=True)

# StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
end_time = time.time()
print(x_train)
print(f"Preprocessing1 execution time: {end_time - start_time} seconds")

          Packet Length Std   Packet Length Variance   Max Packet Length  \
149275          1665.012433              2772266.401                5792   
1925513           46.556417                 2167.500                 132   
1687180            3.000000                    9.000                   6   
2004772            0.000000                    0.000                   6   
2231888            0.000000                    0.000                   6   
...                     ...                      ...                 ...   
2214399           54.224533                 2940.300                 136   
458549           539.975795               291573.859                1460   
1438146            0.000000                    0.000                   2   
212694          2163.472947              4680615.192                5792   
663591            17.527122                  307.200                  63   

          Average Packet Size   Avg Bwd Segment Size   Destination Port  \
149275      

In [11]:
start_time = time.time()

# SMOTE
smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train_scaled, y_train)
print(np.unique(x_train_smote))

end_time = time.time()
print("Unique classes in y_train:", np.unique(y_train))
print("Class distribution in y_train:", pd.Series(y_train).value_counts())
print(f"Preprocessing2 execution time: {end_time - start_time} seconds")

[-992.32701316 -661.55101487 -496.16236746 ...  280.1671507  1213.65174524
 1214.62737693]
Unique classes in y_train: ['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator' 'Web Attack � Brute Force' 'Web Attack � Sql Injection'
 'Web Attack � XSS']
Class distribution in y_train:  Label
BENIGN                        1818477
DoS Hulk                       184858
PortScan                       127144
DDoS                           102421
DoS GoldenEye                    8234
FTP-Patator                      6350
SSH-Patator                      4718
DoS slowloris                    4637
DoS Slowhttptest                 4399
Bot                              1573
Web Attack � Brute Force         1206
Web Attack � XSS                  522
Infiltration                       29
Web Attack � Sql Injection         17
Heartbleed                          9
Name: count, dtype: int64
Preprocessing2 ex

In [14]:
# Start time measurement
start_time = time.time()

# Train the RF model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_smote, y_train_smote)

# Make a prediction on the test data
x_test_scaled = scaler.transform(x_test)
y_pred = rf_model.predict(x_test_scaled)

# End time measurement
end_time = time.time()
# Print the elapsed time
print(f"Traning execution time: {end_time - start_time} seconds")

Traning execution time: 4511.077584028244 seconds


In [15]:
# Evaluate the model
print(confusion_matrix(y_test, y_pred))

print(np.unique(y_test))
print(classification_report(y_test, y_pred, target_names=np.unique(y_test)))

[[446912    236     19      2    177     17      1    215      0      0
     291      0     37   3626   3087]
 [     9    384      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     6      0  25594      0      6      0      0      0      0      0
       0      0      0      0      0]
 [     1      0      0   2057      0      1      0      0      0      0
       0      0      0      0      0]
 [     2      0      0     56  46137      0      0      0      0      0
       0      0      0      0     20]
 [     0      0      0      1      0   1085      5      0      0      0
       0      0      1      4      4]
 [     1      0      0      0      0      1    826      0      0      0
       0      0      0      1    330]
 [     0      0      0      0      0      0      0   1587      0      0
       0      0      0      0      1]
 [     0      0      0      0      0      0      0      0      2      0
       0      0      0      0      0]
 [     1  